In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import os.path

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, scene

## South of Redmond

In [ ]:
product_urlpath = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"
measurement_group = "IW/VV"
dem_urlpath = "South-of-Redmond-10m.tif"
orbit_group = f"{measurement_group}/orbit"
calibration_group = f"{measurement_group}/calibration"
output_urlpath = os.path.basename(product_urlpath) + "-GTC.tif"
interp_method = "nearest"
multilook = None
kwargs = {
    "chunks": 4096,
    "override_product_files": "{dirname}/{prefix}{swath}-{polarization}{ext}",
}

In [ ]:
measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)

beta_nought

In [ ]:
%%time
dem_raster = scene.open_dem_raster(dem_urlpath)
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

In [ ]:
_ = dem_raster.plot()

In [ ]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

In [ ]:
coordinate_conversion = xr.open_dataset(
    product_urlpath,
    engine="sentinel-1",
    group=f"{measurement_group}/coordinate_conversion",
    **kwargs,
)  # type: ignore
ground_range = xarray_sentinel.slant_range_time_to_ground_range(
    acquisition.azimuth_time,
    acquisition.slant_range_time,
    coordinate_conversion,
)
interp_kwargs = {"ground_range": ground_range}
slant_range_time0 = coordinate_conversion.slant_range_time.values[0]

slant_range_time0

In [ ]:
%%time

gtc = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
).compute()

gtc

In [ ]:
gtc.rio.set_crs(dem_raster.rio.crs)
gtc.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
_ = gtc.plot(vmax=0.5)

In [ ]:
dem_normal = scene.compute_diff_normal(dem_ecef)
cos_incidence = xr.dot(dem_normal, -acquisition.dem_direction, dims="axis").compute()
cos_incidence

In [ ]:
%%time

azimuth_time0 = measurement.azimuth_time.values[0]
azimuth_time_interval = measurement.attrs["azimuth_time_interval"]
slant_range_time_interval = measurement.attrs["slant_range_time_interval"]

# compute dem image coordinates
slant_range_index = (
    acquisition.slant_range_time - slant_range_time0
) / slant_range_time_interval / 5

azimuth_index = (
    (acquisition.azimuth_time - azimuth_time0) / geocoding.ONE_SECOND
) / azimuth_time_interval

slant_range_index = np.round(slant_range_index).astype(int)
azimuth_index = np.round(azimuth_index).astype(int)

slant_range_index

In [ ]:
%%time

geocoded = np.maximum(cos_incidence, 0).assign_coords(
    slant_range_index=slant_range_index, azimuth_index=azimuth_index
)

stacked_geocoded = (
    geocoded.stack(z=("y", "x"))
    .reset_index("z")
    .set_index(z=("azimuth_index", "slant_range_index"))
)

grouped = stacked_geocoded.groupby("z")

grouped

In [ ]:
%%time

flat_count = grouped.sum()
flat_count

In [ ]:
%%time

flat_count_smooth = (
    flat_count.unstack("z")
    .fillna(0)
    .rolling(z_level_0=3, z_level_1=3, center=True)
    .mean()
    .stack(z=("z_level_0", "z_level_1"))
)

flat_count_smooth

In [ ]:
%%time

stacked_count = flat_count_smooth.sel(
    z=stacked_geocoded.indexes["z"]
).assign_coords(stacked_geocoded.coords)

weights_cos = stacked_count.set_index(z=("y", "x")).unstack("z")

In [ ]:
_ = weights_cos.plot(vmax=2)

In [ ]:
rtc_cos = gtc / weights_cos
rtc_cos

In [ ]:
rtc_cos.rio.set_crs(dem_raster.rio.crs)
rtc_cos.rio.to_raster(
    output_urlpath.replace("GTC", "RTC-cos"),
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
_ = rtc_cos.plot(vmax=0.5)

In [ ]:
%%time

weights_gamma = geocoding.gamma_weights(
    dem_ecef.compute(),
    acquisition.compute(),
    slant_range_time0=slant_range_time0,
    azimuth_time0=measurement.azimuth_time.values[0],
    azimuth_time_interval=measurement.attrs["azimuth_time_interval"],
    slant_range_time_interval=measurement.attrs["slant_range_time_interval"],
    pixel_spacing_azimuth=measurement.attrs["sar:pixel_spacing_azimuth"],
    pixel_spacing_range=measurement.attrs["sar:pixel_spacing_range"],
)

weights_gamma

In [ ]:
_ = weights_gamma.plot(vmax=1, x="x")

In [ ]:
rtc_gamma = gtc / weights_gamma ** 2
rtc_gamma

In [ ]:
rtc_gamma.rio.set_crs(dem_raster.rio.crs)
rtc_gamma.rio.to_raster(
    output_urlpath.replace("GTC", "RTC-gamma2"),
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
_ = rtc_gamma.plot(vmax=1)